<a href="https://colab.research.google.com/github/culiacanai/Aprende_Python_con_GoogleColab/blob/main/notebooks/14_Proyecto_Final_con_IA.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🚀 Proyecto Final con IA

### Aprende Python con Google Colab — por [Culiacan.AI](https://culiacan.ai)

**Nivel:** 🔴 Avanzado  
**Duración estimada:** 120 minutos  
**Requisitos:** Haber completado los Notebooks 01–13

---

En este notebook vas a:
- Construir un proyecto real de principio a fin que combina **APIs + datos + ML + visualización**
- Consumir APIs reales: OpenAI (GPT), Open-Meteo, ExchangeRate, RestCountries
- Crear un **Asistente de Inteligencia de Negocios** que analiza datos y genera reportes
- Aprender a integrar modelos de lenguaje (LLMs) en tus aplicaciones Python
- Aplicar todo lo aprendido en el curso en un solo proyecto

> 💡 Este es el notebook que junta todo. Vas a construir algo que podrías usar en tu trabajo real.

---

## El Proyecto: Asistente de Inteligencia de Negocios

Vamos a construir un sistema que:
1. **Recopila datos** de múltiples APIs en tiempo real
2. **Analiza** los datos con Pandas y NumPy
3. **Predice** tendencias con Machine Learning
4. **Genera reportes** con texto inteligente usando la API de OpenAI
5. **Visualiza** todo en un dashboard profesional


---

## 0. Preparación

In [ ]:
!pip install openai -q

import os
import json
import requests
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

print("✅ Librerías listas")
print(f"   Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

---

## 1. Configurar las APIs

Vamos a usar 4 APIs reales:

| API | Qué nos da | API Key |
|-----|-----------|---------|
| **Open-Meteo** | Clima en tiempo real y pronóstico | No requiere ✅ |
| **ExchangeRate** | Tipo de cambio actualizado | No requiere ✅ |
| **RestCountries** | Datos de países | No requiere ✅ |
| **OpenAI** | Generación de texto con GPT | Sí requiere 🔑 |

### Configurar la API Key de OpenAI

Para la sección de OpenAI necesitas una API key. Si no la tienes:
1. Crea una cuenta en [platform.openai.com](https://platform.openai.com/)
2. Ve a API Keys y genera una nueva
3. Agrégala como Secret en Colab (menú 🔑 a la izquierda) con el nombre `OPENAI_API_KEY`

> ⚠️ Las secciones sin OpenAI funcionan al 100% sin API key. La sección de OpenAI es opcional pero muy valiosa.


In [ ]:
# Intentar cargar la API key de OpenAI (opcional)
openai_disponible = False

try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    openai_disponible = True
    print("✅ OpenAI API key cargada desde Colab Secrets")
except:
    try:
        OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")
        if OPENAI_API_KEY:
            openai_disponible = True
            print("✅ OpenAI API key cargada desde variable de entorno")
    except:
        pass

if not openai_disponible:
    print("⚠️ OpenAI API key no encontrada")
    print("   Las secciones de OpenAI se saltarán automáticamente")
    print("   Para habilitarlas, agrega tu key en Colab Secrets con nombre OPENAI_API_KEY")

# Configurar cliente OpenAI si está disponible
if openai_disponible:
    from openai import OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)

---

## 2. Módulo de recopilación de datos

### 2.1 API de Clima: Open-Meteo

In [ ]:
def obtener_clima(ciudad: str, lat: float, lon: float, dias: int = 7) -> dict:
    """Obtiene clima actual y pronóstico de Open-Meteo."""
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat, "longitude": lon,
        "current": "temperature_2m,relative_humidity_2m,wind_speed_10m,apparent_temperature,weather_code",
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max",
        "timezone": "America/Mazatlan", "forecast_days": dias,
    }
    try:
        resp = requests.get(url, params=params, timeout=10)
        datos = resp.json()
        return {"ciudad": ciudad, "actual": datos.get("current", {}),
                "diario": datos.get("daily", {}), "status": "ok"}
    except Exception as e:
        return {"ciudad": ciudad, "status": "error", "error": str(e)}

# Probar
ciudades_sinaloa = {
    "Culiacán": (24.8049, -107.3940), "Mazatlán": (23.2494, -106.4111),
    "Los Mochis": (25.7908, -108.9939), "Guasave": (25.5667, -108.4667),
}

datos_clima = {}
for ciudad, (lat, lon) in ciudades_sinaloa.items():
    datos_clima[ciudad] = obtener_clima(ciudad, lat, lon)
    time.sleep(0.3)

for ciudad, datos in datos_clima.items():
    if datos["status"] == "ok":
        actual = datos["actual"]
        print(f"🌡️ {ciudad}: {actual.get('temperature_2m', 'N/A')}°C, "
              f"Humedad: {actual.get('relative_humidity_2m', 'N/A')}%, "
              f"Viento: {actual.get('wind_speed_10m', 'N/A')} km/h")
    else:
        print(f"❌ {ciudad}: {datos.get('error', 'Error desconocido')}")

### 2.2 API de Tipo de Cambio: ExchangeRate

In [ ]:
def obtener_tipo_cambio(base: str = "MXN") -> dict:
    """Obtiene tipos de cambio actualizados."""
    url = f"https://open.er-api.com/v6/latest/{base}"
    try:
        resp = requests.get(url, timeout=10)
        datos = resp.json()
        if datos.get("result") == "success":
            return {"status": "ok", "base": base, "rates": datos["rates"],
                    "updated": datos.get("time_last_update_utc", "")}
        return {"status": "error", "error": "API returned non-success"}
    except Exception as e:
        return {"status": "error", "error": str(e)}

tc = obtener_tipo_cambio("MXN")
if tc["status"] == "ok":
    monedas = {"USD": "Dólar", "EUR": "Euro", "GBP": "Libra", "CAD": "Dólar canadiense",
               "JPY": "Yen", "BRL": "Real", "COP": "Peso colombiano"}
    print(f"💱 Tipo de cambio ({tc['base']}):")
    for codigo, nombre in monedas.items():
        tasa = tc["rates"].get(codigo, 0)
        inverso = 1 / tasa if tasa else 0
        print(f"  1 {codigo} ({nombre:<20}) = ${inverso:.2f} MXN")

### 2.3 API de Países: RestCountries

In [ ]:
def obtener_datos_pais(nombre: str) -> dict:
    """Obtiene datos de un país desde RestCountries."""
    url = f"https://restcountries.com/v3.1/name/{nombre}"
    try:
        resp = requests.get(url, timeout=10)
        if resp.status_code == 200:
            datos = resp.json()[0]
            moneda = list(datos.get("currencies", {}).values())
            return {
                "status": "ok", "nombre": datos["name"]["common"],
                "nombre_oficial": datos["name"].get("official", ""),
                "capital": datos.get("capital", ["N/A"])[0],
                "region": datos.get("region", ""),
                "subregion": datos.get("subregion", ""),
                "poblacion": datos.get("population", 0),
                "area_km2": datos.get("area", 0),
                "moneda": moneda[0]["name"] if moneda else "N/A",
                "idiomas": list(datos.get("languages", {}).values()),
                "bandera": datos.get("flag", ""),
            }
        return {"status": "error", "error": f"HTTP {resp.status_code}"}
    except Exception as e:
        return {"status": "error", "error": str(e)}

paises = ["Mexico", "Brazil", "Chile", "Argentina", "Colombia"]
datos_paises = []
for pais in paises:
    datos = obtener_datos_pais(pais)
    if datos["status"] == "ok":
        datos_paises.append(datos)
        print(f"{datos['bandera']} {datos['nombre']}: {datos['poblacion']:,} hab, Capital: {datos['capital']}")
    time.sleep(0.3)

df_paises = pd.DataFrame(datos_paises)
df_paises[["nombre", "capital", "poblacion", "area_km2", "moneda"]]

### 2.4 API de OpenAI: Generación de texto con GPT

Esta es la API que le da "inteligencia" a nuestro asistente. GPT puede analizar datos y generar reportes en lenguaje natural.


In [ ]:
def generar_texto(prompt: str, system: str = "Eres un analista de datos experto. Respondes en español, de forma concisa y profesional.", max_tokens: int = 500) -> str:
    """Genera texto usando la API de OpenAI."""
    if not openai_disponible:
        return "[OpenAI no disponible — configura tu API key para habilitar esta función]"
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": prompt},
            ],
            max_tokens=max_tokens, temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"[Error al llamar OpenAI: {e}]"

respuesta = generar_texto("En una oración, ¿qué es Machine Learning?")
print(f"🤖 GPT dice: {respuesta}")

---

## 3. Análisis de datos combinados

In [ ]:
# Construir DataFrame de clima
registros_clima = []
for ciudad, datos in datos_clima.items():
    if datos["status"] == "ok":
        actual = datos["actual"]
        diario = datos["diario"]
        registro = {
            "ciudad": ciudad, "temp_actual": actual.get("temperature_2m"),
            "sensacion": actual.get("apparent_temperature"),
            "humedad": actual.get("relative_humidity_2m"),
            "viento": actual.get("wind_speed_10m"),
        }
        if diario:
            temps_max = [t for t in diario.get("temperature_2m_max", []) if t is not None]
            temps_min = [t for t in diario.get("temperature_2m_min", []) if t is not None]
            lluvias = [p for p in diario.get("precipitation_sum", []) if p is not None]
            if temps_max:
                registro["temp_max_prom"] = np.mean(temps_max)
                registro["temp_min_prom"] = np.mean(temps_min)
                registro["lluvia_total_7d"] = sum(lluvias)
                registro["dias_lluvia"] = sum(1 for p in lluvias if p > 0)
        registros_clima.append(registro)

df_clima = pd.DataFrame(registros_clima)
print("📊 Datos de clima consolidados:")
df_clima

In [ ]:
# Visualización de clima
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
colores = ["#2E86AB", "#F18F01", "#A23B72", "#C73E1D"]

ax = axes[0, 0]
if not df_clima.empty:
    bars = ax.bar(df_clima["ciudad"], df_clima["temp_actual"], color=colores[:len(df_clima)])
    for bar, temp in zip(bars, df_clima["temp_actual"]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
                f"{temp:.1f}°C", ha="center", fontweight="bold")
ax.set_title("🌡️ Temperatura Actual", fontweight="bold")
ax.set_ylabel("°C")

ax = axes[0, 1]
if not df_clima.empty:
    bars = ax.bar(df_clima["ciudad"], df_clima["humedad"], color=colores[:len(df_clima)])
    for bar, hum in zip(bars, df_clima["humedad"]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f"{hum:.0f}%", ha="center", fontweight="bold")
ax.set_title("💧 Humedad", fontweight="bold")
ax.set_ylabel("%")

ax = axes[1, 0]
if "temp_max_prom" in df_clima.columns:
    x = range(len(df_clima))
    ax.bar(x, df_clima["temp_max_prom"] - df_clima["temp_min_prom"],
           bottom=df_clima["temp_min_prom"], color=colores[:len(df_clima)], alpha=0.7)
    ax.set_xticks(x)
    ax.set_xticklabels(df_clima["ciudad"])
ax.set_title("🌤️ Rango Temperatura (7 días)", fontweight="bold")
ax.set_ylabel("°C")

ax = axes[1, 1]
if "lluvia_total_7d" in df_clima.columns:
    bars = ax.bar(df_clima["ciudad"], df_clima["lluvia_total_7d"], color=colores[:len(df_clima)])
    for bar, lluvia in zip(bars, df_clima["lluvia_total_7d"]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f"{lluvia:.1f}mm", ha="center", fontweight="bold")
ax.set_title("🌧️ Lluvia Total (7 días)", fontweight="bold")
ax.set_ylabel("mm")

for a in axes.flat:
    a.spines["top"].set_visible(False)
    a.spines["right"].set_visible(False)

plt.suptitle("Dashboard Climático — Sinaloa en Tiempo Real", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

---

## 4. Predicción con Machine Learning

In [ ]:
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error

gapminder = px.data.gapminder()
gm = gapminder.copy()
gm["log_pib"] = np.log10(gm["gdpPercap"])
gm["log_pop"] = np.log10(gm["pop"])
continentes_dummies = pd.get_dummies(gm["continent"], prefix="cont", drop_first=True)
gm = pd.concat([gm, continentes_dummies], axis=1)

feature_cols = ["log_pib", "log_pop", "year"] + list(continentes_dummies.columns)
X = gm[feature_cols].values
y = gm["lifeExp"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"🤖 Modelo entrenado: Gradient Boosting")
print(f"   R²:  {r2:.3f}")
print(f"   MAE: {mae:.2f} años")

importancias = pd.Series(modelo.feature_importances_, index=feature_cols).sort_values(ascending=False)
print(f"\n📊 Top features:")
for feat, imp in importancias.head(5).items():
    print(f"   {feat}: {imp:.3f}")

In [ ]:
def predecir_esperanza_vida(pib_per_capita, poblacion, anio=2025, continente="Americas"):
    """Predice esperanza de vida usando el modelo entrenado."""
    log_pib = np.log10(pib_per_capita)
    log_pop = np.log10(poblacion)
    features = {"log_pib": log_pib, "log_pop": log_pop, "year": anio}
    for col in feature_cols:
        if col.startswith("cont_"):
            features[col] = 1 if col == f"cont_{continente}" else 0
    X_pred = np.array([[features[col] for col in feature_cols]])
    return modelo.predict(X_pred)[0]

paises_prediccion = {
    "México": {"pib": 10_000, "pop": 130_000_000, "cont": "Americas"},
    "Brasil": {"pib": 9_000, "pop": 215_000_000, "cont": "Americas"},
    "Chile": {"pib": 17_000, "pop": 19_500_000, "cont": "Americas"},
    "Argentina": {"pib": 10_000, "pop": 46_000_000, "cont": "Americas"},
    "Colombia": {"pib": 6_500, "pop": 52_000_000, "cont": "Americas"},
    "Japón": {"pib": 40_000, "pop": 125_000_000, "cont": "Asia"},
    "Nigeria": {"pib": 2_200, "pop": 220_000_000, "cont": "Africa"},
    "Alemania": {"pib": 48_000, "pop": 84_000_000, "cont": "Europe"},
}

print("🔮 Predicciones de Esperanza de Vida (2025):")
print(f"{'País':<15} {'PIB/cáp':>10} {'Población':>14} {'Predicción':>12}")
print("-" * 55)
predicciones = {}
for pais, datos in paises_prediccion.items():
    pred = predecir_esperanza_vida(datos["pib"], datos["pop"], 2025, datos["cont"])
    predicciones[pais] = pred
    print(f"{pais:<15} ${datos['pib']:>9,} {datos['pop']:>13,} {pred:>10.1f} años")

---

## 5. Análisis What-If: ¿Qué pasaría si...?

In [ ]:
pib_base = 10_000
poblacion_mx = 130_000_000
escenarios_pib = np.arange(3000, 50001, 1000)
predicciones_escenario = [predecir_esperanza_vida(pib, poblacion_mx, 2025, "Americas") for pib in escenarios_pib]

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(escenarios_pib, predicciones_escenario, color="#2E86AB", linewidth=2.5)
ax.fill_between(escenarios_pib, predicciones_escenario, alpha=0.1, color="#2E86AB")

ev_actual = predecir_esperanza_vida(pib_base, poblacion_mx, 2025, "Americas")
ax.axvline(pib_base, color="#C73E1D", linestyle="--", linewidth=2, alpha=0.7)
ax.annotate(f"México actual\n${pib_base:,}\n{ev_actual:.1f} años",
            xy=(pib_base, ev_actual), xytext=(pib_base + 5000, ev_actual - 3),
            arrowprops=dict(arrowstyle="->", color="#C73E1D"),
            fontsize=10, color="#C73E1D", fontweight="bold")

ev_doble = predecir_esperanza_vida(pib_base * 2, poblacion_mx, 2025, "Americas")
ax.axvline(pib_base * 2, color="#27AE60", linestyle="--", linewidth=2, alpha=0.7)
ax.annotate(f"PIB ×2\n${pib_base*2:,}\n{ev_doble:.1f} años",
            xy=(pib_base * 2, ev_doble), xytext=(pib_base * 2 + 3000, ev_doble + 2),
            arrowprops=dict(arrowstyle="->", color="#27AE60"),
            fontsize=10, color="#27AE60", fontweight="bold")

ax.set_title("🇲🇽 ¿Cómo afecta el PIB a la esperanza de vida de México?", fontsize=14, fontweight="bold")
ax.set_xlabel("PIB per cápita (USD)")
ax.set_ylabel("Esperanza de vida predicha (años)")
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

ganancia = ev_doble - ev_actual
print(f"\n📊 Si México duplicara su PIB per cápita:")
print(f"   Esperanza de vida pasaría de {ev_actual:.1f} a {ev_doble:.1f} años (+{ganancia:.1f})")

---

## 6. Generación de reportes con IA (OpenAI)

Aquí es donde la IA generativa hace su magia: toma todos los datos que recopilamos y genera un reporte en lenguaje natural.

In [ ]:
# Preparar contexto con datos recopilados
contexto_datos = f'''
DATOS RECOPILADOS EN TIEMPO REAL:

1. CLIMA EN SINALOA:
{df_clima.to_string() if not df_clima.empty else 'No disponible'}

2. TIPO DE CAMBIO:
- 1 USD = ${1/tc["rates"]["USD"]:.2f} MXN
- 1 EUR = ${1/tc["rates"]["EUR"]:.2f} MXN

3. PREDICCIONES DE ESPERANZA DE VIDA (modelo ML con R²={r2:.3f}):
{json.dumps(predicciones, indent=2, ensure_ascii=False)}

4. ANÁLISIS WHAT-IF:
- México actual: PIB $10,000 → {ev_actual:.1f} años de esperanza de vida
- Si duplica PIB: $20,000 → {ev_doble:.1f} años (+{ganancia:.1f} años)
'''

prompt_reporte = f'''Basándote en los siguientes datos reales recopilados hoy, genera un reporte ejecutivo breve (máximo 300 palabras) con:
1. Resumen del clima en Sinaloa
2. Situación del tipo de cambio
3. Insight sobre esperanza de vida en LATAM
4. Recomendación basada en el análisis what-if de México

{contexto_datos}

Formato: usa bullets y sé directo. Incluye números específicos.
'''

print("📝 Generando reporte con IA...")
reporte = generar_texto(prompt_reporte, max_tokens=800)
print("\n" + "=" * 65)
print("  REPORTE EJECUTIVO — INTELIGENCIA DE NEGOCIOS")
print("  Generado con Python + APIs + ML + GPT")
print(f"  {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("=" * 65)
print(reporte)

In [ ]:
# Asistente que responde preguntas sobre los datos
def preguntar_al_asistente(pregunta: str) -> str:
    """Hace una pregunta al asistente de BI."""
    system = "Eres un asistente de Inteligencia de Negocios para Ver de Verdad, una cadena de ópticas en Sinaloa, México. Respondes en español con datos específicos."
    prompt = f"Datos disponibles:\n{contexto_datos}\n\nPregunta: {pregunta}"
    return generar_texto(prompt, system=system, max_tokens=300)

preguntas = [
    "¿Es buen momento para importar armazones de Europa considerando el tipo de cambio?",
    "¿Cómo está el clima para abrir una nueva sucursal en Mazatlán esta semana?",
    "¿Qué país de LATAM tiene mejor pronóstico de esperanza de vida y por qué?",
]

for pregunta in preguntas:
    print(f"\n❓ {pregunta}")
    respuesta = preguntar_al_asistente(pregunta)
    print(f"🤖 {respuesta}")
    print("-" * 60)

---

## 7. Dashboard final integrado

In [ ]:
fig = plt.figure(figsize=(18, 14))
fig.patch.set_facecolor("#FAFAFA")
fig.suptitle("DASHBOARD DE INTELIGENCIA DE NEGOCIOS\nVer de Verdad — Datos en Tiempo Real",
             fontsize=20, fontweight="bold", y=0.99)

# KPIs
ax_kpi = fig.add_axes([0.05, 0.94, 0.9, 0.03])
ax_kpi.axis("off")
usd_mxn = 1 / tc["rates"]["USD"] if tc["status"] == "ok" else 0
temp_cln = df_clima[df_clima["ciudad"] == "Culiacán"]["temp_actual"].values[0] if not df_clima.empty else 0
kpi_text = (f"💵 USD/MXN: ${usd_mxn:.2f}    |    🌡️ Culiacán: {temp_cln:.1f}°C    |    "
            f"🤖 Modelo ML: R²={r2:.3f}    |    📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")
ax_kpi.text(0.5, 0.5, kpi_text, ha="center", va="center", fontsize=12)

# 1. Clima
ax1 = fig.add_subplot(2, 3, 1)
colores_c = ["#2E86AB", "#F18F01", "#A23B72", "#C73E1D"]
if not df_clima.empty:
    bars = ax1.bar(df_clima["ciudad"], df_clima["temp_actual"], color=colores_c[:len(df_clima)])
    for bar, temp in zip(bars, df_clima["temp_actual"]):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
                f"{temp:.1f}°", ha="center", fontweight="bold", fontsize=9)
ax1.set_title("🌡️ Temperatura", fontweight="bold")
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)

# 2. Tipo de cambio
ax2 = fig.add_subplot(2, 3, 2)
mons = ["USD", "EUR", "GBP", "CAD"]
vals = [1/tc["rates"][m] for m in mons]
bars = ax2.bar(mons, vals, color=colores_c)
for bar, val in zip(bars, vals):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
            f"${val:.2f}", ha="center", fontweight="bold", fontsize=9)
ax2.set_title("💱 Tipo de Cambio (→ MXN)", fontweight="bold")
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

# 3. Predicciones de esperanza de vida
ax3 = fig.add_subplot(2, 3, 3)
paises_ev = list(predicciones.keys())
valores_ev = list(predicciones.values())
orden = np.argsort(valores_ev)
colores_ev = ["#2E86AB" if p == "México" else "#A4C3D2" for p in np.array(paises_ev)[orden]]
ax3.barh(np.array(paises_ev)[orden], np.array(valores_ev)[orden], color=colores_ev)
for i, (p, v) in enumerate(zip(np.array(paises_ev)[orden], np.array(valores_ev)[orden])):
    ax3.text(v + 0.2, i, f"{v:.1f}", va="center", fontsize=9)
ax3.set_title("🔮 Esperanza de Vida (ML)", fontweight="bold")
ax3.spines["top"].set_visible(False)
ax3.spines["right"].set_visible(False)

# 4. What-If México
ax4 = fig.add_subplot(2, 3, 4)
ax4.plot(escenarios_pib, predicciones_escenario, color="#2E86AB", linewidth=2)
ax4.axvline(pib_base, color="#C73E1D", linestyle="--", alpha=0.7, label="Actual")
ax4.axvline(pib_base * 2, color="#27AE60", linestyle="--", alpha=0.7, label="PIB ×2")
ax4.fill_between(escenarios_pib, predicciones_escenario, alpha=0.1, color="#2E86AB")
ax4.set_title("📈 What-If: PIB vs Esperanza (MX)", fontweight="bold")
ax4.legend(fontsize=8)
ax4.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x/1000:.0f}K"))
ax4.spines["top"].set_visible(False)
ax4.spines["right"].set_visible(False)

# 5. Importancia de features
ax5 = fig.add_subplot(2, 3, 5)
top_imp = importancias.head(6).sort_values()
ax5.barh(top_imp.index, top_imp.values, color="#A23B72")
ax5.set_title("🎯 Features ML", fontweight="bold")
ax5.spines["top"].set_visible(False)
ax5.spines["right"].set_visible(False)

# 6. Info
ax6 = fig.add_subplot(2, 3, 6)
ax6.axis("off")
info = ["📊 FUENTES DE DATOS", "", "🌡️ Open-Meteo (clima)", "💱 ExchangeRate (TC)",
        "🌎 RestCountries (países)", f"🤖 Scikit-learn (R²={r2:.3f})",
        "🧠 OpenAI GPT (reportes)", "", "Culiacan.AI"]
ax6.text(0.05, 0.95, "\n".join(info), transform=ax6.transAxes, fontsize=10,
         verticalalignment="top", fontfamily="monospace",
         bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))

plt.tight_layout(rect=[0, 0.01, 1, 0.93])
fig.savefig("datos/dashboard_bi.png", dpi=200, bbox_inches="tight", facecolor="#FAFAFA")
print("✅ Dashboard guardado como datos/dashboard_bi.png")
plt.show()

---

## 🔥 Retos

1. **Agregar más APIs:** Integra una API de noticias (como NewsAPI) o la API de Wikipedia para agregar contexto a tu reporte. Haz que GPT integre las noticias relevantes al análisis.

2. **Sistema de alertas:** Crea un sistema que revise el clima y tipo de cambio cada cierto tiempo (simula con un loop) y envíe alertas cuando la temperatura supere 40°C o el dólar suba más de un 2%.

3. **Asistente conversacional:** Construye un loop interactivo donde el usuario pueda hacer preguntas en lenguaje natural y el asistente responda usando los datos recopilados.


In [ ]:
# Reto 1: Agregar más APIs
# Tu código aquí 👇


In [ ]:
# Reto 2: Sistema de alertas
# Tu código aquí 👇


In [ ]:
# Reto 3: Asistente conversacional
# Tu código aquí 👇


---

## 📋 Resumen: lo que construimos

### Arquitectura del sistema

```
         APIs externas                    ML + IA
  ┌──────────────────────┐      ┌──────────────────┐
  │ Open-Meteo (clima)   │      │ Scikit-learn (ML) │
  │ ExchangeRate (TC)    │─────▶│ OpenAI GPT (NLP)  │
  │ RestCountries (datos)│      └────────┬─────────┘
  └──────────┬───────────┘               │
             │                           ▼
             ▼                  ┌──────────────────┐
  ┌──────────────────────┐      │   Dashboard +     │
  │  Pandas + NumPy      │─────▶│   Reporte IA      │
  │  (análisis)          │      └──────────────────┘
  └──────────────────────┘
```

### Tecnologías usadas en el proyecto

| Tecnología | Notebook donde la aprendimos |
|-----------|----------------------------|
| Pandas | 07 — Pandas Básico |
| Matplotlib | 08 — Visualización |
| APIs + JSON | 10 — APIs y JSON |
| NumPy | 11 — Intro a NumPy |
| Análisis de datos | 12 — Análisis de Datos Real |
| Machine Learning | 13 — Intro a Machine Learning |
| OpenAI API | 14 — Este notebook |

---

## ⏭️ ¿Qué sigue?

En el último notebook del curso obtendrás tu guía de **Próximos Pasos** en tu camino con Python e IA.

👉 [15 — Próximos Pasos](15_Proximos_Pasos.ipynb)

---

<p align="center">
  Hecho con ❤️ por <a href="https://culiacan.ai">Culiacan.AI</a> — Culiacán reconocida en el mundo por su talento y emprendimiento en Inteligencia Artificial
</p>
